In [1]:
import pandas as pd
from pandas import Series, DataFrame
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import numpy as np
folder = "/data/PDGA_MAP4/"
from Levenshtein import distance as lev_dist
import os
import pickle
import random
from collections import Counter
from operator import itemgetter
import joblib
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from scipy.stats import rankdata
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn import svm
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from rdkit import Chem
from rdkit.Chem.rdmolfiles import MolToSmiles, MolFromSmiles
from rdkit.Chem import Lipinski, Descriptors
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
import tmap as tm
from faerun import Faerun
from map4 import MAP4Calculator
from tqdm.notebook import tqdm

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
df_all = pd.read_pickle(folder+"pickles/all_sequences_with_NN_prop_helicity-hem.pkl")

In [3]:
df_selected = pd.concat([pd.read_csv("data/pdga_filtered_clustered.csv"),pd.read_csv("data/pdga_rnd.csv")])

In [4]:
df_selected["isPredNotHemolytic"] = 1
df_selected["isPredActive"] = 1

In [5]:
selected_seqs = df_selected.Sequence.to_list()

In [6]:
len(df_all)

726002

In [7]:
df_10k = df_all.sample(10000, random_state=0)

In [8]:
df_10k = pd.concat([df_10k, df_selected])

In [9]:
df_10k = df_10k.drop_duplicates("Sequence")

In [10]:
MAP4 = MAP4Calculator(dimensions=1024)
def calc_map4(smiles):
    mol = Chem.MolFromSmiles(smiles)
    smiles = Chem.MolToSmiles(mol, isomericSmiles=False)
    mol = Chem.MolFromSmiles(smiles)
    map4 = MAP4.calculate(mol)
    return np.array(map4)

In [11]:
MAP4_shingles = MAP4Calculator(dimensions=1024, return_strings=True)
def calc_map4_shingles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    smiles = Chem.MolToSmiles(mol, isomericSmiles=False)
    mol = Chem.MolFromSmiles(smiles)
    map4 = MAP4_shingles.calculate(mol)
    return set(map4)

In [12]:
df_10k["MAP4"] = df_10k.SMILES.parallel_map(calc_map4)

In [13]:
df_10k["MAP4_shingles"] = df_10k.SMILES.parallel_map(calc_map4_shingles)

In [24]:
df_10k_ = df_10k[["isPredNotHemolytic", "isPredActive", "length", "JD", "HydroMoment", "fraction_PredHelical", "Sequence", "MAP4_shingles", "MAP4"]]
df_10k = df_10k_.dropna().copy()

In [45]:
def is_selected(seq):
    if seq in selected_seqs:
        return 1
    else:
        return 0
    
def size(seq):
    if seq in selected_seqs:
        return 2
    else:
        return 1

In [46]:
df_10k["Size"] = df_10k.Sequence.map(size)
df_10k["Selected"] = df_10k.Sequence.map(is_selected)

In [27]:
if not os.path.exists(folder+"PDGA_anticancer_LSHforest"):
    fps = df_10k["MAP4"].tolist()
    fps_ = []

    for fp in fps:
        fp_ = tm.VectorUint(fp)
        fps_.append(fp_)

    lf = tm.LSHForest(1024, 32)
    lf.batch_add(fps_)

    lf.index()
    lf.store(folder + 'PDGA_anticancer_LSHforest')
else:
    lf = tm.LSHForest(1024, 32)
    lf.restore(folder + 'PDGA_anticancer_LSHforest')

In [49]:
if not os.path.exists(folder+"PDGA_anticancer_sim_metric.pkl"):
    fps = df_10k["MAP4_shingles"].tolist()
    similarity_metric = np.zeros((len(fps), len(fps)))
    np.fill_diagonal(similarity_metric, 1.0)
    for i, a in enumerate(tqdm(fps)):
        for j in range(i):
            b = fps[j]
            similarity_metric[i,j] = len(a.intersection(b))/len(a.union(b))
            similarity_metric[j,i] = similarity_metric[i,j]
    joblib.dump(similarity_metric, folder+"PDGA_anticancer_sim_metric.pkl")
else:
    similarity_metric = joblib.load(folder+"PDGA_anticancer_sim_metric.pkl")

KeyboardInterrupt: 

In [28]:
if not os.path.exists(folder+"tm_layout.pkl"):
    cfg = tm.LayoutConfiguration()

    # config
    cfg.node_size = 1 / 35
    cfg.mmm_repeats = 2
    cfg.sl_extra_scaling_steps = 5
    cfg.k = 20
    cfg.sl_scaling_type = tm.RelativeToAvgLength
    # Compute the layout

    x_, y_, s, t, gp = tm.layout_from_lsh_forest(lf, cfg)
    tm_layout = {"x": list(x_), "y": list(y_), "s" : list(s), "t" : list(t)}
    joblib.dump(tm_layout, folder+"tm_layout.pkl")
else:
    tm_layout = joblib.load(folder+"tm_layout.pkl")

In [29]:
df_10k.isnull().values.any()

False

In [47]:
isPredNonHemolytic = df_10k.isPredNotHemolytic.tolist()
isPredActive = df_10k.isPredActive.tolist()
length = df_10k.length.tolist()
JD = df_10k.JD.tolist()
HydroMoment = df_10k.HydroMoment.tolist()
fraction_PredHelical = df_10k.fraction_PredHelical.tolist()
labels = df_10k.Sequence.tolist()
selected = df_10k.Selected.tolist()
Size = df_10k.Size.to_list()

In [33]:
custom_cmap = ListedColormap([ 'lightgray', 'limegreen'], name="custom")

groups = ["0 - Predicted Hemolytic", "1 - Predicted Non-hemolytic"]
labels_groups, groups = Faerun.create_categories(groups)

groups2 = ["0 - Predicted Inactive", "1 - Predicted Active"]
labels_groups2, groups2 = Faerun.create_categories(groups2)

groups3 = ["0 - No", "1 - Yes"]
labels_groups3, groups3 = Faerun.create_categories(groups3)

In [34]:
len(tm_layout["y"])

9996

In [35]:
df_10k.JD

169317    0.445
300038    0.586
644187    0.415
235141    0.415
95321     0.505
          ...  
5         0.387
6         0.455
7         0.581
8         0.587
9         0.354
Name: JD, Length: 9996, dtype: float64

In [51]:
faerun = Faerun(view="front", coords=False, title='MAP4_PDGA_anticancer')#, clear_color="#ffffff",)
faerun.add_scatter("MAP4_PDGA_anticancer",\
                   {"x": tm.VectorFloat(tm_layout["x"]), \
                        "y": tm.VectorFloat(tm_layout["y"]), \
                        "c": [length, JD, HydroMoment, fraction_PredHelical, isPredNonHemolytic, isPredActive, selected],\
                        "labels": labels,\
                        "s": Size}, \
                    has_legend=True, \
                    colormap=["rainbow", "rainbow_r", "rainbow", "rainbow", custom_cmap, custom_cmap, custom_cmap], \
                    point_scale=5, \
                    categorical=[False, False, False, False, True, True, True, True],\
                    series_title=["length", "True JD from VNWKKILGKIIKVVK-NH2", "Hydrophbic Moment", "Predicted Helical Faction", \
                                  "Predicted Hemolycity", "Predicted Activity", "Selected for Synthesis"], \
                    max_legend_label=[str(round(max(length),2)), str(round(max(JD),2)), str(round(max(HydroMoment),2)), \
                                      str(round(max(fraction_PredHelical),2)),\
                                      None, None, None],\
                    min_legend_label=[str(round(min(length),2)), str(round(min(JD),2)), str(round(min(HydroMoment),2)), str(round(min(fraction_PredHelical),2)),\
                                   None, None, None],\
                    legend_labels=[None, None, None, None, labels_groups, labels_groups2, labels_groups3])

faerun.add_tree("MAP4_PDGA_anticancer_tree", {"from": tm.VectorUint(tm_layout["s"]), "to": tm.VectorUint(tm_layout["t"])}, point_helper="MAP4_PDGA_anticancer", color="aaaaaa")
faerun.plot('MAP4_PDGA_anticancer')

/home/alice/Code/PDGA_MAP4/MAP4_PDGA_anticancer.html